# 🏛️ Pyramid Perspective Correction - Teotihuacan

This notebook transforms aerial pyramid images to create a frontal view perspective using OpenCV perspective correction.

**Created for Google Colab** 📱

---

In [ ]:
# Install required packages (if not already installed)
!pip install opencv-python matplotlib numpy pillow

# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
from PIL import Image
import os

print("✅ All packages installed and imported successfully!")

In [ ]:
def upload_image():
    """
    Upload an image file in Google Colab
    """
    print("📸 Please upload your Teotihuacan pyramid image:")
    uploaded = files.upload()
    
    # Get the uploaded filename
    filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {filename}")
    
    return filename

# Upload your image
image_filename = upload_image()

In [ ]:
# Load and display the uploaded image
img = cv2.imread(image_filename)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
plt.title("📷 Original Teotihuacan Pyramid Image", fontsize=16, pad=20)
plt.axis('off')
plt.show()

print(f"📐 Image dimensions: {img_rgb.shape[1]} x {img_rgb.shape[0]} pixels")

In [ ]:
def interactive_point_selection(image_path):
    """
    Interactive function to help select the correct source points
    by clicking on the image.
    """
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    points = []
    
    def onclick(event):
        if event.inaxes and len(points) < 4:
            points.append([event.xdata, event.ydata])
            plt.plot(event.xdata, event.ydata, 'ro', markersize=12)
            plt.text(event.xdata + 15, event.ydata - 15, f'{len(points)}', 
                    fontsize=16, color='red', weight='bold', 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.9))
            plt.draw()
            
            if len(points) == 4:
                # Draw lines connecting the points
                x_coords = [p[0] for p in points] + [points[0][0]]
                y_coords = [p[1] for p in points] + [points[0][1]]
                plt.plot(x_coords, y_coords, 'r-', linewidth=3, alpha=0.7)
                plt.draw()
                print("✅ All 4 points selected!")
                print("📍 Points coordinates:")
                for i, point in enumerate(points):
                    labels = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
                    print(f"  {i+1}. {labels[i]}: [{point[0]:.1f}, {point[1]:.1f}]")
    
    fig, ax = plt.subplots(figsize=(14, 10))
    ax.imshow(img_rgb)
    ax.set_title("🎯 Click to select 4 corners of the pyramid base\n" +
                "Order: 1️⃣ top-left, 2️⃣ top-right, 3️⃣ bottom-right, 4️⃣ bottom-left", 
                fontsize=16, pad=20)
    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()
    
    return np.float32(points) if len(points) == 4 else None

# Run interactive point selection
print("📍 Select the pyramid base corners by clicking on the image")
print("⚠️ Make sure to select in order: top-left → top-right → bottom-right → bottom-left")
selected_points = interactive_point_selection(image_filename)

In [ ]:
def correct_pyramid_perspective(image_path, src_points_custom=None, output_filename="corrected_pyramid.jpg"):
    """
    Transform the perspective of a pyramid image to make it appear as if 
    the focal plane is parallel to the pyramid's front face.
    """
    
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Could not load image. Check the file path.")
    
    # Convert BGR to RGB for matplotlib display
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get image dimensions
    height, width = img.shape[:2]
    
    # Use custom points if provided, otherwise use default points
    if src_points_custom is not None:
        src_points = np.float32(src_points_custom)
        print("🎯 Using custom selected points")
    else:
        # Define default source points (you can adjust these if needed)
        src_points = np.float32([
            [width * 0.35, height * 0.45],  # Top-left of pyramid base
            [width * 0.65, height * 0.45],  # Top-right of pyramid base  
            [width * 0.75, height * 0.85],  # Bottom-right of pyramid base
            [width * 0.25, height * 0.85]   # Bottom-left of pyramid base
        ])
        print("⚠️ Using default points (you may want to adjust these)")
    
    # Define destination points (rectangular perspective)
    margin = 100
    dst_points = np.float32([
        [margin, margin],                    # Top-left
        [width - margin, margin],            # Top-right
        [width - margin, height - margin],   # Bottom-right
        [margin, height - margin]            # Bottom-left
    ])
    
    # Calculate the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Apply the perspective transformation
    corrected_img = cv2.warpPerspective(img_rgb, matrix, (width, height))
    
    # Save the corrected image
    corrected_bgr = cv2.cvtColor(corrected_img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_filename, corrected_bgr)
    
    return corrected_img, matrix, src_points, dst_points

# Apply perspective correction using selected points
try:
    if selected_points is not None and len(selected_points) == 4:
        corrected_img, transformation_matrix, src_pts, dst_pts = correct_pyramid_perspective(
            image_filename, 
            selected_points, 
            "corrected_pyramid.jpg"
        )
        print("✅ Perspective correction applied successfully!")
    else:
        print("❌ No valid points selected. Using default points...")
        corrected_img, transformation_matrix, src_pts, dst_pts = correct_pyramid_perspective(
            image_filename, 
            None, 
            "corrected_pyramid.jpg"
        )
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Display the results side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Original image with source points marked
axes[0].imshow(img_rgb)
axes[0].plot([src_pts[0][0], src_pts[1][0], src_pts[2][0], src_pts[3][0], src_pts[0][0]], 
             [src_pts[0][1], src_pts[1][1], src_pts[2][1], src_pts[3][1], src_pts[0][1]], 
             'r-', linewidth=3, label='Source area', alpha=0.8)
axes[0].scatter(src_pts[:, 0], src_pts[:, 1], c='red', s=120, zorder=5, edgecolors='white', linewidth=2)

# Add point labels
labels = ['1', '2', '3', '4']
for i, point in enumerate(src_pts):
    axes[0].text(point[0] + 20, point[1] - 20, labels[i], fontsize=16, color='red', 
                weight='bold', bbox=dict(boxstyle="circle,pad=0.3", facecolor="white", alpha=0.9))

axes[0].set_title('📷 Original Image with Source Points', fontsize=16, pad=20)
axes[0].legend(fontsize=12)
axes[0].axis('off')

# Corrected image with destination points marked
axes[1].imshow(corrected_img)
axes[1].plot([dst_pts[0][0], dst_pts[1][0], dst_pts[2][0], dst_pts[3][0], dst_pts[0][0]], 
             [dst_pts[0][1], dst_pts[1][1], dst_pts[2][1], dst_pts[3][1], dst_pts[0][1]], 
             'g-', linewidth=3, label='Destination area', alpha=0.8)
axes[1].scatter(dst_pts[:, 0], dst_pts[:, 1], c='green', s=120, zorder=5, edgecolors='white', linewidth=2)

axes[1].set_title('🎯 Perspective Corrected Image', fontsize=16, pad=20)
axes[1].legend(fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("🎉 Transformation complete!")
print(f"📁 Corrected image saved as: corrected_pyramid.jpg")

In [ ]:
# Download the corrected image
def download_corrected_image():
    """
    Download the corrected image to your local machine
    """
    try:
        files.download('corrected_pyramid.jpg')
        print("✅ Download started! Check your Downloads folder.")
    except Exception as e:
        print(f"❌ Download error: {e}")

# Download the corrected image
download_corrected_image()

print("💾 Your perspective-corrected pyramid image has been downloaded!")

In [ ]:
def fine_tune_perspective_correction(image_path, src_points, scale_factor=0.8):
    """
    Fine-tune the perspective correction with additional options
    """
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width = img.shape[:2]
    
    # Calculate centered destination rectangle
    center_x, center_y = width // 2, height // 2
    dst_width = int(width * scale_factor)
    dst_height = int(height * scale_factor)
    
    dst_points = np.float32([
        [center_x - dst_width//2, center_y - dst_height//2],   # Top-left
        [center_x + dst_width//2, center_y - dst_height//2],   # Top-right
        [center_x + dst_width//2, center_y + dst_height//2],   # Bottom-right
        [center_x - dst_width//2, center_y + dst_height//2]    # Bottom-left
    ])
    
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    corrected_img = cv2.warpPerspective(img_rgb, matrix, (width, height))
    
    return corrected_img, matrix

# Fine-tune with different scale factors
if selected_points is not None:
    scale_factors = [0.6, 0.8, 1.0]
    
    fig, axes = plt.subplots(1, len(scale_factors), figsize=(18, 6))
    
    for i, scale in enumerate(scale_factors):
        fine_tuned_img, _ = fine_tune_perspective_correction(image_filename, selected_points, scale)
        axes[i].imshow(fine_tuned_img)
        axes[i].set_title(f'Scale Factor: {scale}', fontsize=14)
        axes[i].axis('off')
    
    plt.suptitle('🔧 Fine-Tuned Perspective Corrections', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("🎛️ Try different scale factors to find the best result!")
    print("💡 Lower scale factors = more zoomed view, Higher = more of the image visible")
else:
    print("⚠️ Please run the point selection cell first!")

## 📋 Usage Instructions

### 🚀 **Quick Start Guide:**

1. **📸 Upload Image** (Cell 2)
   - Run Cell 2 and upload your Teotihuacan pyramid image
   - Supported formats: JPG, PNG, etc.

2. **👀 Preview Image** (Cell 3)
   - View your uploaded image and check dimensions

3. **🎯 Select Points** (Cell 4)
   - Click on the image to select 4 corners of the pyramid base
   - **Order is critical**: top-left → top-right → bottom-right → bottom-left
   - Points should form a quadrilateral around the pyramid base

4. **⚙️ Apply Correction** (Cell 5)
   - Automatically applies perspective correction using your selected points
   - Falls back to default coordinates if no points selected

5. **📊 View Results** (Cell 6)
   - Compare original vs. corrected images side by side
   - Red markers show source area, green shows destination

6. **💾 Download** (Cell 7)
   - Download the corrected image to your computer

7. **🔧 Fine-Tune** (Cell 8, Optional)
   - Experiment with different scale factors for optimal results

---

### 💡 **Pro Tips:**

- **Point Selection**: Choose corners that clearly define the pyramid's base rectangle
- **Re-run Flexibility**: You can re-run any cell to try different settings
- **Best Results**: Works optimally when pyramid occupies significant image area
- **Troubleshooting**: If correction looks distorted, try selecting different points

---

### 🔧 **Troubleshooting:**

- **Point selection not working?** → Ensure you're clicking within the image boundaries
- **Distorted results?** → Check point selection order (top-left → top-right → bottom-right → bottom-left)
- **Severe distortion?** → Try adjusting destination rectangle size in the code
- **Upload issues?** → Verify image format (JPG, PNG) and file size

---

### 🎯 **Expected Results:**

The perspective correction transforms your aerial pyramid view into a frontal perspective, making it appear as if you're viewing the pyramid straight-on with the focal plane parallel to its front face.

**Enjoy your perspective-corrected Teotihuacan pyramid! 🏛️✨**